### Get images and normalize them

In [2]:
print("lorem ipsum")

lorem ipsum


### Apply Data augmentation 
#### (Different data aug can be applied to test several models)

In [3]:
print("lorem ipsum")

lorem ipsum


### Split Data

In [4]:
print("lorem ipsum")

lorem ipsum


### Print Data and explain them (part of the project)

In [5]:
print("lorem ipsum")

lorem ipsum


### Load pretrain coco 2017 model (detect stop sign auto)

In [6]:
print("lorem ipsum")

lorem ipsum


### Train

In [7]:
print("lorem ipsum")

lorem ipsum


### Samples predicts

In [8]:
print("lorem ipsum")

lorem ipsum


### Analysis of result

In [9]:
print("lorem ipsum")

lorem ipsum


### Store Data for later (to compare results of data augs Tier 1, 2, 3, ...)

In [10]:
print("lorem ipsum")

lorem ipsum
